<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../_static/logo_stacked.png" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 8. Multi-page app</h2></div>
<br><br>

In [ ]:
import panel as pn

pn.extension()

# Multi-page app

More often that not you will want to serve more than a simple app, either because your app grew in complexity or because you already had some big ideas in mind! Multi-page apps can be defined in two ways that look similar but are actually quite different:

1. serving *multiple apps*
2. serving one app with *multiple "pages"*

We will explore these options, the first one being straightforward and the second one a little more involved but with simple approaches available.

## Multiple apps

Panel can easily serve multiple apps (i.e. files) with `panel serve file1.py file2.py file3.py ...`. Each app will be served on its own endpoint, you will be able to visit them at an URL ending with `/file1.py`, `/file3.py`, `/file3.py`, etc. In this case the landing page will be a built-in index page with each app having its own card.

You can define the app that is going to be the landing page with the `--index` flag, with for instance `panel serve home.py app.py --index home.py`. You may want to dynamically redirect your users to another served app, which you can do manipulating `pn.state.location` as in the example below. Alternatively you could render an HTML pane with a links to your other apps, e.g. `pn.pane.HTML('<a href="/app">Visit app</a>')`.

```python
# home.py
import panel as pn

def redirect(_):
    pn.state.location.pathname = '/app'
    pn.state.location.reload = True

button = pn.widgets.Button(name='Visit app')
button.on_click(redirect)

pn.Column(
    '# Landing page',
    button,
).servable()
```

## One app with multiple "pages"

While the *multiple apps* option introduced above is pretty straightforward, it doesn't make it very easy to update one app from another, it's actually pretty hard. When you serve a single app, managing its state and interactivity is a lot easier. So we will see how you can turn one app in an app that behaves like a multi-page app.

### Tabs

Using the `Tabs` layout to reproduce the behavior of a multipage app is the simplest option available, at the expense of a user interface that might not be quite what we were looking for. Even if you will not end up with this solution, it's the good way to to get a working prototype.

Try the app below and check that updating the slider on the *Advanced* page correctly updates the result.

In [ ]:
w_a = pn.widgets.FloatSlider(name='a')
w_b = pn.widgets.FloatSlider(name='b', value=1)
w_c = pn.widgets.FloatSlider(name='c', value=0.5)


def dummy_algo(a, b, c):
    return round((a*10 + b)**c, 2)

main = pn.Column(
    '### Algorithm', 'Go to the next page to find more advanced options',
    w_a, w_b,
    'Result:', pn.bind(dummy_algo, w_a, w_b, w_c)
)   
advanced = pn.Column('These are the advanced options:', w_c)


pn.Tabs(('Main', main), ('Advanced', advanced))

### Swapping content

The second approach we will see consists of controlling the content of a layout from a widget, updating the content depending on the value of the widget. In the example below, we create such a widget and a placeholder layout, together with a callback watching the widget value and updating the content of the placeholder. We finally put together the app in a `Row`.

In [ ]:
w_control = pn.widgets.RadioBoxGroup(options=['Main', 'Advanced'])
placeholder = pn.Column(main)

@pn.depends(w_control, watch=True)
def swap(control):
    if control == 'Main':
        placeholder[:] = [main]
    elif control == 'Advanced':
        placeholder[:] = [advanced]

side = pn.Column('Pages:', w_control)
pn.Row(side, placeholder, min_height=300)

We can easily make this app prettier by wrapping it in a template. Use the *Preview* functionality to display the app.

In [ ]:
pn.template.BootstrapTemplate(
    title='Multi page',
    sidebar=[side],
    main=[placeholder]
).servable();

To get closer to the kind of multipage app you are used to, you can sync the control widget with a URL query parameter, allowing your users to visit and bookmark a particular "page". The code you will need to do that is simply `pn.state.location.sync(w_control, {'value': 'page'})`, you will be introduced in more details about deep linking in one of the next guides.